In [1]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense, Activation,Dropout
from keras import regularizers
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization

import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

import pandas as pd

Using TensorFlow backend.


In [2]:
#Folder for the dataset
datasetFolder = '/home/carnd/dbpedia2016/step_3x25/dataset/'

#Number of files
numberOfFiles = 638

#Test split
testSplit=0.2

In [3]:
def load_data(datasetFolder, datasetXFile, datasetYFile, wrap=True, printIt=False):
    #print('Loading X')
    # load file
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        cols = head.split(',')
        numberOfCols = len(cols)
        #print(numberOfCols)
        numberOfRows=0
        for line in f:
            numberOfRows+=1
        f.close()
    if(printIt):
        print('Input Features: {} x {}'.format(numberOfRows,numberOfCols))
    if(wrap==True):
        maxY = 8384
    else:
        maxY = numberOfCols-1
    half=(numberOfCols//maxY)*0.5
    dataX = np.zeros([numberOfRows,maxY],np.int8)
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        rowCounter=0
        for line in f:
            row=line.split(',')
            for i in range(1, len(row)):
                if(int(row[i])<=0):
                    continue;
                val = 1 + ((int(row[i])-1)//maxY);
                if(val>half):
                    val = 0 - (val - half)
                dataX[rowCounter][(int(row[i])-1)%maxY]= val
                #if((1 + ((int(row[i])-1)//maxY))>1):
                #    print("{} data[{}][{}] = {}".format(int(row[i])-1, rowCounter,(int(row[i])-1)%maxY,1 + ((int(row[i])-1)//maxY)))
            rowCounter+=1
        f.close()
   
    #print('Loading Y')
    # load file
    with open(datasetFolder + datasetYFile, "r") as f:
        head = f.readline()
        cols = head.split(',')
        numberOfCols = len(cols)
        #print(numberOfCols)
        numberOfRows=0
        for line in f:
            numberOfRows+=1
        f.close()

    if(printIt):
        print('Output Features: {} x {}'.format(numberOfRows,numberOfCols))
    dataY = np.zeros([numberOfRows,(numberOfCols-1)],np.float16)
    with open(datasetFolder + datasetYFile, "r") as f:
        head = f.readline()
        rowCounter=0
        for line in f:
            row=line.split(',')
            for i in range(1, len(row)):
                if(int(row[i])<=0):
                    continue;
                dataY[rowCounter][(int(row[i])-1)]=1
            rowCounter+=1
        f.close()
        

    return dataX, dataY

In [4]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv', printIt=True)

Input Features: 3605 x 413794
Output Features: 3605 x 526


In [5]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv')

In [6]:
print(dataX.shape)
print(dataX[0:5])

(3605, 8384)
[[1 1 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [7]:
print(dataY.shape)
print(dataY[0:5])

(3605, 525)
[[ 0.  1.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [8]:
print("Input Features for classification: {}".format(dataX.shape[1]))
print("Output Classes for classification: {}".format(dataY.shape[1]))

Input Features for classification: 8384
Output Classes for classification: 525


In [9]:
deepModel = Sequential(name='Deep Model (5 Dense Layers)')
deepModel.add(Dense(1024, input_dim=dataX.shape[1], init='glorot_normal'))
deepModel.add(BatchNormalization())
deepModel.add(Activation('relu'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(1024, init='glorot_normal'))
deepModel.add(BatchNormalization())
deepModel.add(Activation('relu'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(1024, init='glorot_normal'))
deepModel.add(BatchNormalization())
deepModel.add(Activation('relu'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(1024, init='glorot_normal'))
deepModel.add(BatchNormalization())
deepModel.add(Activation('relu'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(1024, init='glorot_normal'))
deepModel.add(BatchNormalization())
deepModel.add(Activation('relu'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_normal'))
models = [deepModel]

In [10]:
# Compile model
import keras.backend as K

def count_predictions(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives, predicted_positives, possible_positives

def f1score(y_true, y_pred):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = 2.0 * precision * recall / (precision+recall+ K.epsilon())
    return f1score

def fBetaScore(y_true, y_pred, beta):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = (1+(beta*beta)) * precision * recall / ((beta*beta*precision)+recall+ K.epsilon())
    return f1score

for model in models:
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=[f1score])

In [11]:
def fit_data(model, dataX, dataY):
    # Fit the model
    #model.fit(dataX, dataY, nb_epoch=5, verbose=2, batch_size=256)
    return model.train_on_batch(dataX, dataY)

In [12]:
def countPredictions(y_true, y_pred):
    true_positives = np.sum(np.round(y_pred*y_true))
    predicted_positives = np.sum(np.round(y_pred))
    possible_positives = np.sum(y_true)
    return true_positives, predicted_positives, possible_positives

In [13]:
#Randomize the list of numbers so we can split train and test dataset
listOfFiles=list(range(1,numberOfFiles+1))
import random
random.shuffle(listOfFiles)
splitIndex=int((1-testSplit)*numberOfFiles)

numberOfEons = 5
for model in models:
    for eon in range(0, numberOfEons):
        print('{}. Eon {}/{}'.format(eon+1,eon+1, numberOfEons))
        for trainIndex in range(0,splitIndex):
            dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[trainIndex]), 'datasetY_{}.csv'.format(listOfFiles[trainIndex]))
            #print('Model = {}'.format(model.name))
            #model.fit(dataX, dataY, nb_epoch=1, verbose=0, batch_size=512)
            #sc=model.test_on_batch(dataX,dataY)
            #loss = sc[0]
            #f1score = sc[1]
            loss, f1score=fit_data(model,dataX, dataY)
            print('Learning for file {} / {} : datasetX/Y_{}\t\tloss={:.4f} f1score={:.4f}'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex], loss, f1score), end='\r')

        counts = {} 
    
        counts[model.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
    
        for testIndex in range(splitIndex, numberOfFiles):
            dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
            predY=model.predict_on_batch(dataX)
            true_positives, predicted_positives, possible_positives = countPredictions(dataY, predY)
            counts[model.name]['true_positives'] += true_positives
            counts[model.name]['predicted_positives'] += predicted_positives
            counts[model.name]['possible_positives'] += possible_positives
            print ('Testing for file {} / {} : datasetX/Y_{} - true +ve:{}  pred +ve:{} possible +ve:{}'.format(testIndex+1, numberOfFiles, listOfFiles[testIndex], true_positives,predicted_positives,possible_positives), end='\r')
    
        count = counts[model.name]
        precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
        recall = (count['true_positives'])/(count['possible_positives']+0.0001)
        f1score = 2.0 * precision * recall / (precision+recall+0.0001)
        print(' - Model = {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(model.name, f1score, precision, recall))

1. Eon 1/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8059	 precision = 0.8752 	 recall = 0.746884.0
2. Eon 2/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8225	 precision = 0.8776 	 recall = 0.774184.0
3. Eon 3/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8276	 precision = 0.8719 	 recall = 0.7877584.0
4. Eon 4/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8290	 precision = 0.8701 	 recall = 0.7918584.0
5. Eon 5/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8317	 precision = 0.8729 	 recall = 0.7944584.0
6. Eon 6/10


KeyboardInterrupt: 

==================================================
# step_3x25 (exact)
==================================================
1. Eon 1/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8059	 precision = 0.8752 	 recall = 0.746884.0
2. Eon 2/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8225	 precision = 0.8776 	 recall = 0.774184.0
3. Eon 3/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8276	 precision = 0.8719 	 recall = 0.7877584.0
4. Eon 4/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8290	 precision = 0.8701 	 recall = 0.7918584.0
5. Eon 5/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8317	 precision = 0.8729 	 recall = 0.7944584.0

==================================================
# step_3x25 (upto)
==================================================
1. Eon 1/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8193	 precision = 0.8731 	 recall = 0.771800
2. Eon 2/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8258	 precision = 0.8690 	 recall = 0.786800
3. Eon 3/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8269	 precision = 0.8706 	 recall = 0.787500
4. Eon 4/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8271	 precision = 0.8708 	 recall = 0.787700
5. Eon 5/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8271	 precision = 0.8684 	 recall = 0.789700
6. Eon 6/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8282	 precision = 0.8684 	 recall = 0.791600
7. Eon 7/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8283	 precision = 0.8661 	 recall = 0.793800
8. Eon 8/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8277	 precision = 0.8685 	 recall = 0.790700
9. Eon 9/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8278	 precision = 0.8675 	 recall = 0.791700
10. Eon 10/10
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8283	 precision = 0.8702 	 recall = 0.790400